In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [2]:
pd.set_option('display.max_columns', None)

## Global Food Prices Database (Cleaned)
Updated Global Market Food Prices with 1.048M rows

In [3]:
price_df = pd.read_csv('../raw_data/Global Food Prices Database/wfp_food_prices_database.csv')

/Users/plyck/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [5]:
price_df.isnull().sum()/len(price_df)

adm0_id               0.000000
adm0_name             0.000000
adm1_id               0.000000
adm1_name             0.302027
mkt_id                0.000000
mkt_name              0.000000
cm_id                 0.000000
cm_name               0.000000
cur_id                0.000000
cur_name              0.000000
pt_id                 0.000000
pt_name               0.000000
um_id                 0.000000
um_name               0.000000
mp_month              0.000000
mp_year               0.000000
mp_price              0.000000
mp_commoditysource    1.000000
dtype: float64

In [6]:
#dropping the mp_commoditysource because it's 100% Nan's

#price_df.drop(columns='mp_commoditysource', inplace=True)

In [7]:
price_df['adm1_name'].nunique()

613

In [8]:
price_df['adm1_name'].fillna("Unknown", inplace=True)

In [9]:
price_df['adm1_name'].nunique()

614

In [10]:
price_df.isnull().sum()/len(price_df)

adm0_id               0.0
adm0_name             0.0
adm1_id               0.0
adm1_name             0.0
mkt_id                0.0
mkt_name              0.0
cm_id                 0.0
cm_name               0.0
cur_id                0.0
cur_name              0.0
pt_id                 0.0
pt_name               0.0
um_id                 0.0
um_name               0.0
mp_month              0.0
mp_year               0.0
mp_price              0.0
mp_commoditysource    1.0
dtype: float64

In [11]:
# Introducing new column 'type' where the cm_name (commodity name) is split to make it easier to read by the nutrition API

In [12]:
price_df['type'] = price_df['cm_name'].apply(lambda x: x.split('-')[0])

In [13]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,type
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN,Bread
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN,Bread
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN,Bread
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN,Bread
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN,Bread


In [14]:
#Filtering for year 2020 only

In [15]:
price_2020_df = price_df[price_df['mp_year'] == 2020]

In [16]:
price_2020_df.shape

(165098, 19)

In [17]:
#Dropping unneccesary columns

In [18]:
price_2020_df.drop(columns=['adm0_id', 'adm1_id', 'mkt_id', 'cm_id', 'cur_id', 'cm_name', 'pt_id', 'mp_commoditysource','um_id'], inplace=True)

/Users/plyck/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [19]:
#Removing rows with other units than 'KG' and 'L'.

In [20]:
price_2020_df = price_2020_df.loc[price_2020_df['um_name'].isin(['KG','L'])]

In [21]:
price_2020_df['cur_name'].unique()

array(['AFN', 'AOA', 'ARS', 'AMD', 'BDT', 'INR', 'BYR', 'XOF', 'BTN',
       'BOB', 'BIF', 'KHR', 'CVE', 'XAF', 'COP', 'CDF', 'DJF', 'USD',
       'EGP', 'ETB', 'GMD', 'GHS', 'GNF', 'IDR', 'IRR', 'IQD', 'JPY',
       'JOD', 'KZT', 'KES', 'KGS', 'LAK', 'LBP', 'LRD', 'LYD', 'MWK',
       'SOS', 'MRO', 'MXN', 'MNT', 'MZN', 'MMK', 'NAD', 'NPR', 'NGN',
       'PKR', 'PYG', 'PEN', 'PHP', 'RWF', 'SLL', 'SSP', 'LKR', 'NIS',
       'SZL', 'SYP', 'TJS', 'THB', 'TRY', 'UGX', 'YER', 'ZMW'],
      dtype=object)

In [22]:
# Source Wiki: It was previously known as the new Israeli shekel and the non-official abbreviation of NIS.
# Need to change NIS to ILS (Israeli New Shekel)

In [23]:
price_2020_df['cur_name'] = price_2020_df['cur_name'].apply(lambda x: x.replace('NIS','ILS'))

In [24]:
currencies = price_2020_df['cur_name'].unique()

In [25]:
type(currencies)

numpy.ndarray

In [26]:
currencies = currencies.tolist()

In [27]:
currencies.sort()

In [28]:
len(currencies)

62

In [29]:
#SSP = South Sudanese pound
# This is not in our currency exchange API so must be removed and added later.

In [30]:
currencies.remove("SSP")

In [31]:
len(currencies)

61

In [32]:
#API call to get the conversion rates in the list currencies

In [33]:
api_key = 'e3479b3ac491d8699be3a12b7cf7e51d'

In [34]:
curr = ','.join(currencies)

In [35]:
response = requests.get(f'http://api.currencylayer.com/live?access_key={api_key}&currencies={curr}&format=1')

In [36]:
response.url

'http://api.currencylayer.com/live?access_key=e3479b3ac491d8699be3a12b7cf7e51d&currencies=AFN,AMD,AOA,ARS,BDT,BIF,BOB,BTN,BYR,CDF,COP,CVE,DJF,EGP,ETB,GHS,GMD,GNF,IDR,ILS,INR,IQD,IRR,JOD,JPY,KES,KGS,KHR,KZT,LAK,LBP,LKR,LRD,LYD,MMK,MNT,MRO,MWK,MXN,MZN,NAD,NGN,NPR,PEN,PHP,PKR,PYG,RWF,SLL,SOS,SYP,SZL,THB,TJS,TRY,UGX,USD,XAF,XOF,YER,ZMW&format=1'

In [37]:
response.json()

{'success': True,
 'terms': 'https://currencylayer.com/terms',
 'privacy': 'https://currencylayer.com/privacy',
 'timestamp': 1639383003,
 'source': 'USD',
 'quotes': {'USDAFN': 103.255653,
  'USDAMD': 493.850311,
  'USDAOA': 568.279026,
  'USDARS': 101.526053,
  'USDBDT': 85.476643,
  'USDBIF': 1986.51918,
  'USDBOB': 6.881991,
  'USDBTN': 75.528971,
  'USDBYR': 19600,
  'USDCDF': 2000.497004,
  'USDCOP': 3889.98,
  'USDCVE': 97.531743,
  'USDDJF': 177.436333,
  'USDEGP': 15.747601,
  'USDETB': 48.515145,
  'USDGHS': 6.074681,
  'USDGMD': 52.450201,
  'USDGNF': 9469.214813,
  'USDIDR': 14334.25,
  'USDILS': 3.10183,
  'USDINR': 75.71905,
  'USDIQD': 1456.200409,
  'USDIRR': 42275.000291,
  'USDJOD': 0.709201,
  'USDJPY': 113.552505,
  'USDKES': 112.530045,
  'USDKGS': 84.783602,
  'USDKHR': 4064.060434,
  'USDKZT': 434.563773,
  'USDLAK': 10969.844934,
  'USDLBP': 1507.222532,
  'USDLKR': 201.834603,
  'USDLRD': 141.803315,
  'USDLYD': 4.580274,
  'USDMMK': 1774.135073,
  'USDMNT': 28

In [38]:
conversion_rates = response.json()

In [39]:
conversion_rates = conversion_rates['quotes']

In [40]:
len(conversion_rates), len(currencies)

(61, 61)

In [41]:
# Making a dictionary of the conversion rates

In [42]:
dic_values = []
for x in conversion_rates.keys():
    dic_values.append(conversion_rates[x])

In [43]:
len(dic_values)

61

In [44]:
conversion_rates_dict = dict(zip(currencies,dic_values))

In [45]:
#SSP will be added as 1 USD = 130.26 SSP

In [46]:
conversion_rates_dict['SSP'] = 130.26

In [47]:
# Adding a new column to our data with the conversion rates

In [48]:
price_2020_df['conv_rate'] = list(map(lambda x: conversion_rates_dict[x] , price_2020_df['cur_name']))

In [49]:
price_2020_df['conv_rate'].nunique()

62

In [50]:
# Adding a new column to our data with the price in USD

In [51]:
price_2020_df["usd_rate"] = price_2020_df["mp_price"] / price_2020_df["conv_rate"] 

In [52]:
price_2020_df

,adm0_name,adm1_name,mkt_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate
69,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,50.0000,Bread,103.255653,0.484235
70,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,2,2020,50.0000,Bread,103.255653,0.484235
71,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,3,2020,50.0000,Bread,103.255653,0.484235
72,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,4,2020,50.0000,Bread,103.255653,0.484235
275,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,25.3750,Wheat,103.255653,0.245749
...,...,...,...,...,...,...,...,...,...,...,...,...
1556443,Zambia,Western,Kalabo,ZMW,Retail,KG,2,2020,6.4480,"Maize meal (white, breakfast)",16.022167,0.402442
1556444,Zambia,Western,Kalabo,ZMW,Retail,KG,3,2020,6.6320,"Maize meal (white, breakfast)",16.022167,0.413927
1556445,Zambia,Western,Kalabo,ZMW,Retail,KG,4,2020,6.4400,"Maize meal (white, breakfast)",16.022167,0.401943
1556446,Zambia,Western,Kalabo,ZMW,Retail,KG,5,2020,6.5168,"Maize meal (white, breakfast)",16.022167,0.406736


In [88]:
#To be used by Streamlit website
price_2020_df.to_csv('price_with_USD.csv')

In [53]:
price_2020_df.shape

(128938, 12)

In [54]:
price_2020_df['mp_month'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [55]:
list_of_foods = price_2020_df['type'].unique()

In [56]:
list_of_foods = list_of_foods.tolist()

In [57]:
list_of_foods.sort()

In [58]:
list_of_foods

['Apples ',
 'Apples (dried) ',
 'Apples (red) ',
 'Avocados ',
 'Bananas ',
 'Bananas (imported) ',
 'Bananas (local) ',
 'Bananas (medium size) ',
 'Barley ',
 'Beans ',
 'Beans (black) ',
 'Beans (black, Nayarit) ',
 'Beans (black, Veracruz) ',
 'Beans (black, Zacatecas) ',
 'Beans (black, imported) ',
 'Beans (butter) ',
 'Beans (catarino) ',
 'Beans (cranberry) ',
 'Beans (dry) ',
 'Beans (fava, dry) ',
 'Beans (green, fresh) ',
 'Beans (haricot) ',
 'Beans (kidney red) ',
 'Beans (magnum) ',
 'Beans (niebe) ',
 'Beans (niebe, white) ',
 'Beans (pod) ',
 'Beans (red) ',
 'Beans (red, fresh) ',
 'Beans (string) ',
 'Beans (sugar) ',
 'Beans (white) ',
 'Beans (yardlong, green) ',
 'Beans(mash) ',
 'Beetroots ',
 'Blackberry ',
 'Bread ',
 'Bread (common) ',
 'Bread (first grade flour) ',
 'Bread (high grade flour) ',
 'Bread (pita) ',
 'Bread (wheat) ',
 'Broccoli ',
 'Buckwheat ',
 'Buckwheat grits ',
 'Bulgur ',
 'Bulgur (brown) ',
 'Butter ',
 'Butter (cow milk) ',
 'Butter (goa

In [59]:
len(list_of_foods)

422

In [60]:
list_of_non_food = ['Charcoal ','Corn Soy Blend (CSB++, food aid) ','Cotton ', 'Dishwashing liquid ','Disinfecting solution ','Fuel (Super Petrol) ','Fuel (diesel) ',
 'Fuel (diesel, parallel market) ',
 'Fuel (kerosene) ',
 'Fuel (petrol','Handwash soap ','Laundry detergent ','Laundry soap ','Salt ','Salt (iodised) ','Shampoo ']

In [61]:
len(list_of_non_food)

16

In [62]:
for x in list_of_non_food:
        print(x)
        price_2020_df = price_2020_df.loc[price_2020_df["type"] != x]

Charcoal 
Corn Soy Blend (CSB++, food aid) 
Cotton 
Dishwashing liquid 
Disinfecting solution 
Fuel (Super Petrol) 
Fuel (diesel) 
Fuel (diesel, parallel market) 
Fuel (kerosene) 
Fuel (petrol
Handwash soap 
Laundry detergent 
Laundry soap 
Salt 
Salt (iodised) 
Shampoo 


In [63]:
price_2020_df['type'].nunique()

406

In [64]:
price_2020_df.shape

(119885, 12)

# Drop columns that we don't need

In [65]:
price_2020_df.head()

,adm0_name,adm1_name,mkt_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate
69,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,50.000,Bread,103.255653,0.484235
70,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,2,2020,50.000,Bread,103.255653,0.484235
71,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,3,2020,50.000,Bread,103.255653,0.484235
72,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,4,2020,50.000,Bread,103.255653,0.484235
275,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,25.375,Wheat,103.255653,0.245749


In [66]:
# We don't need the column mp_year. We know its 2020. We don't need the market name, currency name, month and the adm1_name

In [67]:
price_2020_df.drop(columns=['mp_year','mkt_name','adm1_name', 'mp_month','cur_name'], inplace=True)

In [68]:
price_2020_df.head()

,adm0_name,pt_name,um_name,mp_price,type,conv_rate,usd_rate
69,Afghanistan,Retail,KG,50.000,Bread,103.255653,0.484235
70,Afghanistan,Retail,KG,50.000,Bread,103.255653,0.484235
71,Afghanistan,Retail,KG,50.000,Bread,103.255653,0.484235
72,Afghanistan,Retail,KG,50.000,Bread,103.255653,0.484235
275,Afghanistan,Retail,KG,25.375,Wheat,103.255653,0.245749


In [69]:
grouped_multiple = price_2020_df.groupby(['adm0_name', 'type', 'pt_name','um_name' ]).agg({'mp_price': 'mean','conv_rate': 'mean', 'usd_rate': 'mean'})


In [70]:
grouped_multiple.head()

mp_price   conv_rate  \
adm0_name   type                 pt_name um_name                           
Afghanistan Bread                Retail  KG        50.344430  103.255653   
            Oil (cooking)        Retail  KG       103.044485  103.255653   
            Pulses               Retail  KG        95.488971  103.255653   
            Rice (high quality)  Retail  KG        87.189706  103.255653   
            Rice (low quality)   Retail  KG        48.832043  103.255653   

                                                  usd_rate  
adm0_name   type                 pt_name um_name            
Afghanistan Bread                Retail  KG       0.487571  
            Oil (cooking)        Retail  KG       0.997955  
            Pulses               Retail  KG       0.924782  
            Rice (high quality)  Retail  KG       0.844406  
            Rice (low quality)   Retail  KG       0.472924

In [82]:
grouped_multiple.to_csv('grouped_cleaned_data.csv')

In [83]:
test = pd.read_csv('grouped_cleaned_data.csv')

In [87]:
test.head()

,adm0_name,type,pt_name,um_name,mp_price,conv_rate,usd_rate
0,Afghanistan,Bread,Retail,KG,50.344430,103.255653,0.487571
1,Afghanistan,Oil (cooking),Retail,KG,103.044485,103.255653,0.997955
2,Afghanistan,Pulses,Retail,KG,95.488971,103.255653,0.924782
3,Afghanistan,Rice (high quality),Retail,KG,87.189706,103.255653,0.844406
4,Afghanistan,Rice (low quality),Retail,KG,48.832043,103.255653,0.472924


In [85]:
test.shape

(1252, 7)

## Importing Adrians Dataset

In [71]:
food_types = pd.read_csv('../raw_data/Food Types/food_types_v2.csv')

In [72]:
food_types.head()

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
1,Oil (cooking),1.26,3.24,30.9,not found,116,0.0,3.0,144
2,Pulses,17.9,7.14,60.7,not found,393,0.0,107,393
3,Rice (high quality),7.19,3.85,75.5,not found,1.0,not found,10.0,365
4,Rice (low quality),7.19,3.85,75.5,not found,1.0,not found,10.0,365


In [73]:
food_types['food_type'].nunique()

406

In [74]:
grouped_multiple.shape

(1252, 3)

In [75]:
joined_df = pd.merge(grouped_multiple, food_types, how='outer', left_on = 'type', right_on = 'food_type')

In [76]:
joined_df.head()

,mp_price,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,50.344430,103.255653,0.487571,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
1,114.238000,101.526053,1.125209,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
2,1078.371429,580.236179,1.858504,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
3,27.529414,52.450201,0.524868,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
4,52.387667,84.783602,0.617899,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0


In [77]:
joined_df.shape

(1252, 12)